In [73]:
import psycopg2
import pandas as pd
import requests
from datetime import datetime
from datetime import datetime, timedelta

# Replace these values with your TimescaleDB credentials


db_name="tsdb"
db_user="tsdbadmin"
db_password="xus2e6kh8iecv918"
db_host="yxwiwu71o8.rvzh9hqg0d.tsdb.cloud.timescale.com"
db_port="32943"

# Connect to TimescaleDB
conn = psycopg2.connect(
    host=db_host,
    port=db_port,
    user=db_user,
    password=db_password,
    dbname=db_name
)

# Get today's date and time
now = datetime.now()
print(now)

one_day_ago = datetime.now() - timedelta(hours=2)

print(one_day_ago)
# Define the SQL query
sql_query = f"""
    SELECT
        A.user_id,
        A.movie_id,
        A.min_time,
        A.timestamp AS a_timestamp,
        B.recommended_movies,
        B.timestamp AS b_timestamp,
        CASE
            WHEN POSITION(A.movie_id::text IN B.recommended_movies) > 0 THEN 1
            ELSE 0
        END AS flag
    FROM
        (SELECT user_id, movie_id, timestamp, MIN(time) AS min_time
         FROM user_telemetry
         WHERE timestamp >= '{one_day_ago}'
           AND timestamp <= '{now}'
         GROUP BY user_id, movie_id, timestamp) A
    INNER JOIN
        (SELECT DISTINCT ON (user_id)
            user_id,
            recommended_movies,
            timestamp
         FROM movie_recommendation
         WHERE timestamp >= '{one_day_ago}'
           AND timestamp <= '{now}'
         ORDER BY user_id, timestamp) B
    ON A.user_id = B.user_id
    WHERE A.timestamp > B.timestamp order by A.timestamp
"""

# Read the results into a pandas DataFrame
df = pd.read_sql_query(sql_query, conn)



2023-04-24 03:00:36.814409
2023-04-24 01:00:36.814575


In [74]:
df_no_duplicates = df.drop_duplicates(subset='user_id', keep='first')
# Define a function to fetch user information
def get_user_info(user_id):
    url = f"http://128.2.204.215:8080/user/{user_id}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching user info for user_id {user_id}")
        return None

# Loop through user_ids and fetch user information
user_info_list = []
for user_id in df_no_duplicates["user_id"]:
    user_info = get_user_info(user_id)
    if user_info:
        user_info_list.append(user_info)

# Create a new DataFrame with the fetched user information
user_info_df = pd.DataFrame(user_info_list)
df_no_duplicates["user_id"]=df_no_duplicates["user_id"].astype(int)
user_info_df["user_id"]=user_info_df["user_id"].astype(int)
merged_df = df_no_duplicates.merge(user_info_df, on="user_id")

/tmp/ipykernel_3976415/508762235.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates["user_id"]=df_no_duplicates["user_id"].astype(int)


In [75]:
merged_df['top_movie'] = merged_df['recommended_movies'].apply(lambda x: x.split(',')[0])
# Define a function to fetch movie information
def get_movie_info(top_movie):
    url = f"http://128.2.204.215:8080/movie/{top_movie}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching movie info for movie {top_movie}")
        return None

# Loop through top_movies and fetch movie information
movie_info_list = []
movie_info_cache = {}

for top_movie in merged_df["top_movie"]:
    if top_movie!="":
        if top_movie not in movie_info_cache:
            movie_info = get_movie_info(top_movie)
            if movie_info:
                movie_info_cache[top_movie] = {
                    'top_movie': top_movie,
                    'genres': [genre['name'] for genre in movie_info['genres']],
                    'popularity': movie_info['popularity'],
                    'vote_average': movie_info['vote_average']
                }
        
        movie_info_list.append(movie_info_cache[top_movie])

# Create a new DataFrame with the fetched movie information
movie_info_df = pd.DataFrame(movie_info_list)
movie_info_df = movie_info_df.drop_duplicates(subset='top_movie', keep='first')


# Merge the merged_df with the movie_info_df based on top_movie
final_df = merged_df.merge(movie_info_df, on="top_movie")

# Print the final DataFrame
print(final_df.shape)


(21381, 14)


In [76]:
age_bins = [0, 10, 20, 30, 40, 50, 60, 100]
age_labels = ['0-10', '10-20', '30-40', '40-50', '50-60', '60-70', '70+']

# Create age_group column by binning the age column
final_df['age_group'] = pd.cut(final_df['age'], bins=age_bins, labels=age_labels)



In [77]:
final_df.groupby(["top_movie"]).user_id.count().sort_values(ascending=False).head()

top_movie
shrek+2001                       10539
casablanca+1942                  10538
the+shawshank+redemption+1994       88
goodfellas+1990                     45
pulp+fiction+1994                   25
Name: user_id, dtype: int64

# Model Fairness

#### Gender

In [78]:
final_df.groupby(["gender"]).flag.mean()

gender
F    0.576247
M    0.590937
Name: flag, dtype: float64

#### Age

In [79]:
final_df.groupby(["age_group"]).flag.mean()

age_group
0-10     0.487500
10-20    0.564103
30-40    0.588676
40-50    0.591096
50-60    0.576344
60-70    0.651064
70+      0.600000
Name: flag, dtype: float64

In [80]:
final_df.groupby(["occupation"]).flag.mean()

occupation
K-12 student              0.560759
academic/educator         0.588535
artist                    0.600849
clerical/admin            0.631098
college/grad student      0.578689
customer service          0.622951
doctor/health care        0.590909
executive/managerial      0.597304
farmer                    0.600000
homemaker                 0.561062
lawyer                    0.726027
other or not specified    0.577417
programmer                0.568282
retired                   0.583578
sales/marketing           0.609896
scientist                 0.588547
self-employed             0.586552
technician/engineer       0.555184
tradesman/craftsman       0.534351
unemployed                0.627907
writer                    0.621622
Name: flag, dtype: float64

# System Fairness

In [92]:
sql_query_recommendations = f"""
SELECT DISTINCT ON (user_id)
            user_id,
            recommended_movies,
            timestamp
         FROM movie_recommendation
         WHERE timestamp >= '{one_day_ago}'
           AND timestamp <= '{now}'
         ORDER BY user_id,timestamp"""

# Read the results into a pandas DataFrame
df_recommendations = pd.read_sql_query(sql_query_recommendations, conn)

In [94]:
df_recommendations.head()

,user_id,recommended_movies,timestamp
0,10,"shrek+2001, the+battle+of+algiers+1966, the+em...",2023-04-24 01:16:16.123681
1,100006,"shrek+2001, the+battle+of+algiers+1966, the+em...",2023-04-24 02:02:45.271079
2,100081,"casablanca+1942, goodfellas+1990, the+shawshan...",2023-04-24 01:22:59.145727
3,100101,"casablanca+1942, goodfellas+1990, the+shawshan...",2023-04-24 02:53:43.085209
4,100108,"shrek+2001, the+battle+of+algiers+1966, the+em...",2023-04-24 01:05:19.208718


In [95]:
df_recommendations['top_movie'] = df_recommendations['recommended_movies'].apply(lambda x: x.split(',')[0])

# Define a function to fetch movie information
def get_movie_info(top_movie):
    url = f"http://128.2.204.215:8080/movie/{top_movie}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching movie info for movie {top_movie}")
        return None

# Loop through top_movies and fetch movie information
movie_info_list = []
movie_info_cache = {}

for top_movie in df_recommendations["top_movie"]:
    if top_movie!="":
        if top_movie not in movie_info_cache:
            movie_info = get_movie_info(top_movie)
            if movie_info:
                movie_info_cache[top_movie] = {
                    'top_movie': top_movie,
                    'genres': [genre['name'] for genre in movie_info['genres']],
                    'popularity': movie_info['popularity'],
                    'vote_average': movie_info['vote_average']
                }
        
        movie_info_list.append(movie_info_cache[top_movie])

# Create a new DataFrame with the fetched movie information
movie_info_df = pd.DataFrame(movie_info_list)
movie_info_df = movie_info_df.drop_duplicates(subset='top_movie', keep='first')


# Merge the merged_df with the movie_info_df based on top_movie
df_recommendations_final = df_recommendations.merge(movie_info_df, on="top_movie")

# Print the final DataFrame
print(df_recommendations_final.shape)


(71811, 7)


In [97]:
df_recommendations_final['primary_genre'] = df_recommendations_final['genres'].apply(lambda x: x[0] if len(x) > 0 else None)


### Distribution of top recommended movie

In [91]:
df_recommendations_final.groupby(["top_movie"]).user_id.count().sort_values(ascending=False).head(10)

top_movie
casablanca+1942                              35537
shrek+2001                                   35214
the+shawshank+redemption+1994                  305
goodfellas+1990                                175
the+godfather+1972                              89
pulp+fiction+1994                               77
the+sixth+sense+1999                            39
the+lord+of+the+rings+the+two+towers+2002       34
the+battle+of+algiers+1966                      32
tarzan+1999                                     30
Name: user_id, dtype: int64

### Distribution of top recommended movie genre

In [99]:
df_recommendations_final.groupby(["primary_genre"]).user_id.count().sort_values(ascending=False).head(10)

primary_genre
Drama          36239
Adventure      35337
Thriller          89
Mystery           62
Crime             26
Horror            17
Action            11
Documentary        8
Animation          7
War                7
Name: user_id, dtype: int64